the classes will eventually be split up but they are put together for now to get their interactions correct <br>
sources: <br>
events: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-II/ <br>
portfolio: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-V/ <br>
execution handler: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-VI/

In [1]:
import numpy as np
import pandas as pd
import os, sys

from dataclasses import dataclass

In [2]:
quotes_filename = os.path.join("play_data", "XBTUSD_quotes_191214_0434.csv")
trades_filename = os.path.join("play_data", "XBTUSD_trades_191214_0434.csv")

In [3]:
quotes_full = pd.read_csv(
    quotes_filename,
    index_col='recorded',
    parse_dates=['timestamp', 'recorded']
)

trades_full = pd.read_csv(
    trades_filename,
    index_col='received',
    parse_dates=['timestamp', 'received'],
    dtype={"side":"category"}
)

all_data = {
    "quotes": quotes_full,
    "trades": trades_full
}

display( quotes_full.head() )
display( trades_full.head() )

,timestamp,bidSize,bidPrice,askPrice,askSize
recorded,,,,,
2019-12-14 04:34:40.731941+00:00,2019-12-14 04:34:40.024000+00:00,3840427,7251.5,7252.0,701477
2019-12-14 04:34:41.211565+00:00,2019-12-14 04:34:40.410000+00:00,3840391,7251.5,7252.0,701477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.165000+00:00,3840391,7251.5,7252.0,731477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.183000+00:00,3840187,7251.5,7252.0,731277
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.431000+00:00,3840187,7251.5,7252.0,715277


,timestamp,side,size,price
received,,,,
2019-12-14 04:34:40.742081+00:00,2019-12-14 04:34:38.713000+00:00,Sell,2500,7251.5
2019-12-14 04:34:42.499730+00:00,2019-12-14 04:34:42.378000+00:00,Buy,12000,7252.0
2019-12-14 04:34:48.168746+00:00,2019-12-14 04:34:48.044000+00:00,Buy,100000,7252.0
2019-12-14 04:34:48.332053+00:00,2019-12-14 04:34:48.213000+00:00,Buy,116,7252.0
2019-12-14 04:34:50.665331+00:00,2019-12-14 04:34:50.504000+00:00,Buy,2500,7252.0


In [4]:
@dataclass
class FillEvent:
    timestamp : pd.Timestamp
    symbol: str
    exchange: str
    quantity: int  # change in securities held
    fill_cost: float  # change in cash held
        
    def __post_init__(self):
        pass
    

        
class ExecutionHandler:
    
    def __init__(self, future_data):
        self.future_data = future_data  # df of bid/ask prices to calculate market fills with
        
    def send_order(self, order):
        pass
    
    def generate_fill(self, order):  # walks forward in time to see bid/ask prices
        pass

In [5]:
eh = ExecutionHandler(quotes_full)
f = FillEvent(pd.Timestamp.utcnow(), "XBTUSD", "BitMEX", 10, 20)
f

FillEvent(timestamp=Timestamp('2020-01-11 16:09:53.826128+0000', tz='UTC'), symbol='XBTUSD', exchange='BitMEX', quantity=10, fill_cost=20)

In [6]:
class Portfolio:
    def __init__(self, start_time, symbol_list, all_data, initial_capital=10000.0):
        
#         self.queue = queue  # event queue
        self.data = { k: df[:start_time] for k, df in all_data.items() }
        self.symbol_list = symbol_list 
        self.start_time = start_time
        self.initial_capital = initial_capital
        
        self.current_positions = pd.Series(0, index=symbol_list, name=start_time)
        self.current_holdings = self.construct_current_holdings(start_time)  # series
        
        self.all_positions_list = [self.current_positions]  # list of series
        self.all_holdings_list = [self.current_holdings]*10  # list of series
        
    @property
    def all_positions(self):
        return pd.DataFrame(self.all_positions_list)
    @property
    def all_holdings(self):
        return pd.DataFrame(self.all_holdings_list)
    
    def construct_current_holdings(self, time):  # market values of positions
        holdings = {symbol: 0.0 for symbol in symbol_list }
        holdings["capital"] = self.initial_capital
        holdings["commission"] = 0.0
        holdings["total"] = self.initial_capital
        return pd.Series(holdings, name=time)
    
    def update_fill(self, fill):
        pass
        
    def update_signal(self, signal):
        pass
    
symbol_list = "xbtusd", "ethusd"

p = Portfolio(pd.Timestamp.utcnow(), symbol_list, all_data)

p.current_holdings

xbtusd            0.0
ethusd            0.0
capital       10000.0
commission        0.0
total         10000.0
Name: 2020-01-11 16:09:53.844270+00:00, dtype: float64